In [1]:
#!/usr/bin/env python3
import argparse
from importlib import import_module
from itertools import count
import os

import h5py
import json
import numpy as np
from sklearn.metrics import average_precision_score
import tensorflow as tf

import common
import loss

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.utils.np_utils import to_categorical
from keras.layers import LeakyReLU, Concatenate, concatenate, Lambda, UpSampling2D, Add, Input, Dense
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, BatchNormalization

In [2]:
class Args:
    excluder= 'market1501' #, 'diagonal','cuhk03','duke'
    query_dataset = '/home/k/kajal/triplet-reid/data/market1501/market1501_query.csv'
    query_embeddings = '/home/k/kajal/triplet-reid/experiments/mkt_pixel_ce_exp/aug_query_embeddings.h5'
    query_embeddings_adv = '/home/k/kajal/triplet-reid/experiments//MARKET/marketrecon255/test_embeddings.h5'
    gallery_dataset = '/home/k/kajal/triplet-reid/data/market1501/market1501_test.csv'
    gallery_embeddings =  '/home/k/kajal/triplet-reid/experiments/mkt_pixel_ce_exp/aug_gallery_embeddings.h5'
    metric =  'euclidean'
    #filename = 
    batch_size = 18

In [3]:
args=Args()

In [4]:
# Load the query and gallery data from the CSV files.
query_pids, query_fids = common.load_dataset(args.query_dataset, None)
gallery_pids, gallery_fids = common.load_dataset(args.gallery_dataset, None)

# Load the two datasets fully into memory.
with h5py.File(args.query_embeddings, 'r') as f_query:
    query_embs = np.array(f_query['emb'])
    query_embs_dp = np.array(f_query['emb'])
with h5py.File(args.gallery_embeddings, 'r') as f_gallery:
    gallery_embs = np.array(f_gallery['emb'])
    gallery_embs_dp = np.array(f_gallery['emb'])

In [5]:
gallery_embs=gallery_embs[6618:]
gallery_embs_dp=gallery_embs_dp[6618:]
gallery_pids=gallery_pids[6618:]

In [6]:
uni_pids=np.unique(gallery_pids)
print(len(uni_pids))
uni_query_pids=np.unique(query_pids)
len(uni_query_pids)

750


750

In [7]:
print(query_embs.shape)
print(gallery_embs.shape)

(3368, 128)
(13114, 128)


In [8]:
print(query_embs[0],query_embs[1])

[-1.4895324e+00  4.9818420e-01  7.6958352e-01 -7.2742596e-02
  1.2921383e+00 -1.3292663e+00 -6.6451675e-01  2.2089496e-01
 -5.4692543e-01  9.0875167e-01 -2.6830935e+00  1.1928835e+00
 -7.6162750e-01 -1.6739820e+00 -2.6683328e+00 -9.6505082e-01
  2.9969415e-01  2.4369028e+00 -2.5869861e+00  4.9287838e-01
  1.5305965e+00 -1.2275312e+00  3.3783272e-01 -1.3281310e+00
  5.2735442e-01 -3.5020718e-01  1.0744702e+00 -9.5970106e-01
  5.7255012e-01 -1.3847376e+00  1.1607877e+00  1.7695973e+00
 -2.2717962e+00 -3.7673607e+00 -7.1160597e-01 -2.3803020e-01
  2.5717223e+00  1.2124742e+00  2.3082561e+00  1.3672486e-03
 -6.6192621e-01 -2.7707446e-01 -1.0177685e+00  9.3757915e-01
  1.1460953e+00 -5.2870208e-01  7.4943912e-01  1.5137751e-01
 -3.2341409e+00 -3.5842472e-01 -2.7776301e-01  6.1675835e-01
 -2.4815164e+00 -1.1225467e+00  2.1935356e+00  6.2268490e-01
  4.0969354e-01  1.7986505e+00  7.7366823e-01 -2.4093692e-01
  1.6228625e+00  1.6464910e-01 -2.2330117e-01  1.8286045e+00
  3.3835578e+00  9.22889

In [9]:
#query_embs_dp=query_embs
query_embs_dp.shape
query_embs[0]
print(np.min(query_embs),np.mean(query_embs),np.max(query_embs),np.std(query_embs))
print(np.min(gallery_embs),np.mean(gallery_embs),np.max(gallery_embs),np.std(gallery_embs))

-7.690467 -0.0011640474 6.3633947 1.3787516
-7.29783 -0.007016901 7.0519843 1.3921114


In [10]:
query_embs_dp[0,1]=np.random.normal(0,2.312213)
query_embs_dp=np.exp(0.5) * query_embs_dp +0.01
gallery_embs_dp[0,1]=np.random.normal(0,2.3505316)
gallery_embs_dp=np.exp(0.5) * gallery_embs_dp +0.01

In [11]:
def l1(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.max(np.subtract(actual,pred)).mean()

def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.max(np.square(np.subtract(actual,pred)).mean())

D0 = query_embs + np.random.normal(0,1.3787516)
D1 = query_embs_dp + np.random.normal(0,1.3787516)

D2 = gallery_embs + np.random.normal(0,1.3921114)
D3 = gallery_embs_dp + np.random.normal(0,1.3921114)
#print(D0,D1)
#sens1 = l1(query_embs,query_embs_dp)
sensq = mse(D0,D1)
sensg = mse(D2,D3)
print(sensq,sensg)

1.3578779 14.685372


In [12]:
query_data = query_pids
query_values = array(query_data)
print('values', query_values)
# integer encode
query_label_encoder = LabelEncoder()
query_integer_encoded = query_label_encoder.fit_transform(query_values)
print('encoded', query_integer_encoded)
# binary encode
query_onehot_encoder = OneHotEncoder(sparse=False)
query_integer_encoded = query_integer_encoded.reshape(len(query_integer_encoded), 1)
query_onehot_encoded = query_onehot_encoder.fit_transform(query_integer_encoded)
print(query_onehot_encoded.shape)

values ['0001' '0001' '0001' ... '1501' '1501' '1501']
encoded [  0   0   0 ... 749 749 749]
(3368, 750)


In [13]:
data = gallery_pids
values = array(data)
print('values', values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print('encoded', integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

values ['0001' '0001' '0001' ... '1501' '1501' '1501']
encoded [  0   0   0 ... 749 749 749]
(13114, 750)


In [14]:
X_train  = D3
Y_train_label = onehot_encoded
X_test = D0
Y_test_label = query_onehot_encoded
#print(X_train.size)
#print(X_test.size)
#print(Y_train_label)
#print(Y_test_label)

In [15]:
#print(Y_train_label[2:5000])
#y_train_label = to_categorical(Y_train_label)
#y_train_label.shape

In [16]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
#model.add( Dense(units=512, input_dim=128, kernel_initializer='normal', activation='relu') )
#model.add( Dropout(0.5))
model.add( Dense(units=750, input_dim=128, kernel_initializer='normal', activation='softmax') )
print( model.summary() )
model.compile( loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 750)               96750     
                                                                 
Total params: 96,750
Trainable params: 96,750
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
train_history = model.fit(x=X_train, y=Y_train_label, validation_split=0.2, epochs=1, batch_size=128, verbose=2)

82/82 - 10s - loss: 4.7693 - accuracy: 0.2103 - val_loss: 11.6540 - val_accuracy: 0.0000e+00 - 10s/epoch - 126ms/step


In [18]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test_label)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

Evaluate on test data
106/106 [==============================] - 2s 21ms/step - loss: 4.9341 - accuracy: 0.2233
test loss, test acc: [4.934057712554932, 0.22327791154384613]
Generate predictions for 3 samples
106/106 [==============================] - 2s 16ms/step
predictions shape: [[4.2770654e-02 1.3071230e-03 1.4002275e-04 ... 5.2871305e-04
  2.2377255e-03 4.1954595e-04]
 [7.1114744e-03 2.0610988e-03 1.4104680e-04 ... 3.2420503e-04
  9.6199545e-04 7.9970557e-04]
 [3.5686854e-02 8.7615493e-04 9.3118288e-05 ... 3.7962216e-04
  2.3793653e-03 3.4696434e-04]
 ...
 [1.0126997e-03 3.6603364e-03 9.2211738e-04 ... 1.4890432e-03
  1.8014945e-03 1.5465692e-03]
 [5.8010884e-04 4.9895034e-03 8.4596995e-04 ... 1.2242500e-03
  2.3740786e-03 1.7316269e-03]
 [5.7673722e-04 2.3656713e-03 8.8875985e-04 ... 1.3285979e-03
  2.5859769e-03 1.6622964e-03]]


In [ ]:
from diffprivlib.models import GaussianNB
clf = GaussianNB(epsilon=1,bounds=(-50e50, 50e50))
clf.fit(X_train, gallery_pids)

In [ ]:
clf.predict(X_test)
print("Test accuracy: %f" % clf.score(X_test, query_pids))

In [ ]:
def Hbeta(D=np.array([]), beta=1.0):
    """
        Compute the perplexity and the P-row for a specific value of the
        precision of a Gaussian distribution.
    """

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values in such a way that each
        conditional Gaussian has the same perplexity.
    """

    # Initialize some variables
    print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        if i % 500 == 0:
            print("Computing P-values for point %d of %d..." % (i, n))

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax = np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2.
                else:
                    beta[i] = (beta[i] + betamax) / 2.
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2.
                else:
                    beta[i] = (beta[i] + betamin) / 2.

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
    return P

def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """

    print("Preprocessing the data using PCA...")
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:, 0:no_dims])
    return Y
def tsne(X=np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
        Runs t-SNE on the dataset in the NxD array X to reduce its
        dimensionality to no_dims dimensions. The syntaxis of the function is
        `Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 10
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 200
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4.									# early exaggeration
    P = np.maximum(P, 1e-12)
    

    # Run iterations
    for iter in range(max_iter):

        # Compute pairwise affinities
        sum_Y = np.sum(np.square(Y), 1)
        num = -2. * np.dot(Y, Y.T)
        num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
        num[range(n), range(n)] = 0.
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dY[i, :] = np.sum(np.tile(PQ[:, i] * num[:, i], (no_dims, 1)).T * (Y[i, :] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0.) != (iY > 0.)) + \
                (gains * 0.8) * ((dY > 0.) == (iY > 0.))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        if (iter + 1) % 10 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration %d: error is %f" % (iter + 1, C))

        # Stop lying about P-values
        if iter == 100:
            P = P / 4.

    # Return solution
    return Y
             

In [ ]:
import pylab
X = gallery_embs[500:700]
lbl=list(map(int,gallery_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n1.pdf')
pylab.show()

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test_label)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

In [ ]:
import pylab
X = query_embs[500:700]
lbl=list(map(int,query_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n2.pdf')
pylab.show()

In [ ]:
    #file = query_embs
    #filel = query_pids
    X = query_embs[0:50]
    lbl=list(map(int,query_pids))
    labels = lbl[0:50]#query_pids[0:20]
    print(query_embs,query_pids)
    #X = np.loadtxt("mnist2500_X.txt")
    pylab.figure()
    Y = tsne(X, 2, 50, 20.0)
    plt=pylab.scatter(Y[:, 0],Y[:,1],30,labels,marker='+')
    pylab.axis('off')
    pylab.legend('on')
    pylab.savefig('rsltn3n1.pdf')
    pylab.show()

In [ ]:
    #file = query_embs
    #filel = query_pids
    X = query_embs[0:50]
    lbl=list(map(int,query_pids))
    labels = lbl[0:50]#query_pids[0:20]
    print(query_embs,query_pids)
    #X = np.loadtxt("mnist2500_X.txt")
    pylab.figure()
    Y = tsne(X, 2, 50, 20.0)
    plt=pylab.scatter(Y[:, 0],Y[:,1],30,labels,marker='+')
    pylab.axis('off')
    pylab.legend('on')
    pylab.savefig('rsltn3n1.pdf')
    pylab.show()